In [ ]:
#Maps the NIC LEI population to GLEIF
#Exploration of matching methods for institutions that have only name and address in NIC but may have an LEI


In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#set display options for pandas dataframes
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
#get lei map to panel from
#load NIC ID walk
nic_ids = pd.read_csv("/Users/roellk/Desktop/HMDA/hmda-panel/2017/output/panel_lei_map.txt", sep="|", dtype="object")
nic_ids.fillna("-1", inplace=True)
print(len(nic_ids))
nic_ids

5852


,institution_id,agency_code,filing_period,activity_year,respondent_id,institution_type,cra,email_domain_1,email_domain_2,email_domain_3,respondent_name,respondent_state_code,respondent_city,respondent_state_fips_number,hmda_filer_flag,parent_respondent_id,parent_id_rssd,parent_name,parent_city,parent_state,assets,other_lender_code,top_holder_id_rssd,top_holder_name,top_holder_city,top_holder_state,top_holder_country,id_rssd,id_tax,id_lei,nm_lgl,city,state_cd,zip_cd,street_line1,street_line2
0,546991,5,2018,2017,63201,6,False,frontierccu.org,-1,-1,FRONTIER COMMUNITY CREDIT UNION,KS,LEAVENWORTH,20,False,-1,-1,-1,-1,-1,123326,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,858528,2,2018,2017,858528,2,True,bomva.com,-1,-1,"BANK OF MARION, THE",VA,MARION,51,True,-1,1132391,"SOUTHWEST VIRGINIA BANKSHARES, INC.",MARION,VA,359543,0,1132391,"SOUTHWEST VIRGINIA BANKSHARES, INC.",MARION,VA,UNITED STATES,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,3603765,3,2018,2017,58697,3,False,adamsdairybank.com,-1,-1,ADAMS DAIRY BANK,MO,BLUE SPRINGS,29,False,-1,4564724,"ADAMS DAIRY BANCSHARES, INC.",BLUE SPRINGS,MO,107778,0,4564724,"ADAMS DAIRY BANCSHARES, INC.",BLUE SPRINGS,MO,UNITED STATES,3603765,261585535,549300XVXVMZ9CDXV530,ADAMS DAIRY BANK,BLUE SPRINGS,29,64014,651 NORTHEAST CORONADO DRIVE,0
3,213538,3,2018,2017,11782,3,False,bankofabbeville.com,-1,-1,BANK OF ABBEVILLE & TRUST COMPANY,LA,ABBEVILLE,22,True,-1,-1,-1,-1,-1,180990,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,724490,5,2018,2017,61879,6,False,ticreditunion.org,-1,-1,T AND I CREDIT UNION,MI,CLAWSON,26,True,-1,-1,-1,-1,-1,74289,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,2871099,3,2018,2017,35417,3,False,tristar.bank,-1,-1,TRISTAR BANK,TN,DICKSON,47,True,-1,3442795,"TRISTAR BANCSHARES, INC.",DICKSON,TN,253185,0,3442795,"TRISTAR BANCSHARES, INC.",DICKSON,TN,UNITED STATES,2871099,562136525,549300LTKCJV903ZQR25,TRISTAR BANK,DICKSON,47,37055,719 EAST COLLEGE STREET,0
6,978239,3,2018,2017,10045,3,False,cornerstonebankwv.com,-1,-1,"CORNERSTONE BANK, INC.",WV,WEST UNION,54,True,-1,3217537,"CORNERSTONE FINANCIAL SERVICES, INC.",WEST UNION,WV,169368,0,3217537,"CORNERSTONE FINANCIAL SERVICES, INC.",WEST UNION,WV,UNITED STATES,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,3875151,7,2018,2017,770575554,14,False,santacruzhomefinance.com,-1,-1,SANTA CRUZ HOME FINANCE,CA,SANTA CRUZ,6,True,-1,-1,-1,-1,-1,-1,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,3872039,7,2018,2017,382749215,14,False,vandykmortgage.com,-1,-1,VANDYK MORTGAGE CORPORATION,MI,GRAND RAPIDS,26,True,-1,-1,-1,-1,-1,-1,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,835444,3,2018,2017,11310,3,False,bankofherrin.com,-1,-1,"BANK OF HERRIN, THE",IL,HERRIN,17,True,-1,2527024,"FGH BANCORP, INC.",HERRIN,IL,255647,0,2527024,"FGH BANCORP, INC.",HERRIN,IL,UNITED STATES,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [3]:
#load GLEIF LEI list
gleif_df = pd.read_csv("../output/lei_info.txt", sep="|", dtype="object")
fill_cols = ["legal_city", "legal_name", "hq_city", "hq_country"]
gleif_df.fillna("-1", inplace=True)
gleif_df = gleif_df[]

,hq_city,hq_country,hq_street1,hq_street2,id_lei,legal_city,legal_country,legal_name,legal_post,legal_region,legal_street1,legal_street2
0,NG-LA,NG,27 GERRARD ROAD,IKOYI,029200013A5N6ZD0F605,LAGOS,NG,AFRINVEST SECURITIES LIMITED,23401,NG-LA,27 GERRARD ROAD,IKOYI
1,NG-LA,NG,"9TH FLOOR BOOKSHOP HOUSE, 50/52 BROAD STREET",P.O. BOX 702 LAGOS,029200137F2K8AH5C573,LAGOS,NG,CAPITAL ASSETS LIMITED,23401,NG-LA,"9TH FLOOR BOOKSHOP HOUSE, 50/52 BROAD STREET",P.O. BOX 702 LAGOS
2,NG-LA,NG,"3RD FLOOR, UNTL BUILDING, 1, DAVIES STREET","OFF MARINA, LAGOS. P.O.BOX 1362, MARINA",029200038B4L4ZI1E579,LAGOS,NG,CAPITAL BANCORP PLC,23401,NG-LA,"3RD FLOOR, UNTL BUILDING, 1, DAVIES STREET","OFF MARINA, LAGOS. P.O.BOX 1362, MARINA"
3,NG-LA,NG,"PLOT 1626 C-E, SAKA JOJO STREET","OFF IDEJO STREET, VICTORIA ISLAND",029200048F0K9UI5A508,LAGOS,NG,CAPITAL EXPRESS SECURITIES LTD,23401,NG-LA,"PLOT 1626 C-E, SAKA JOJO STREET","OFF IDEJO STREET, VICTORIA ISLAND"
4,NG-LA,NG,"5, OKOTIE-EBOH STREET",IKOYI,029200222D9S2BC3J917,LAGOS,NG,CARDINALSTONE SECURITIES LIMITED,23401,NG-LA,"5, OKOTIE-EBOH STREET",IKOYI


In [5]:
gleif_df.legal_country.unique()

array(['NG', 'BM', 'SK', 'BG', 'HU', 'RU', 'LT', 'RO', 'CZ', 'GB', 'EE',
       'SI', 'LU', 'NZ', 'LV', 'TR', 'FI', 'JE', 'KY', 'IE', 'DE', 'SE',
       'ES', 'CH', 'IL', 'NL', 'AT', 'SG', 'DK', 'AE', 'HK', 'PL', 'CY',
       'VG', 'BS', 'GR', 'MT', 'NO', 'MY', 'GG', 'ZA', 'KR', 'PT', 'CA',
       'KN', 'US', 'IT', 'MH', 'UG', 'BE', 'GI', 'FR', 'IM', 'PA', 'BZ',
       'LI', 'SC', 'HR', 'CN', 'MC', 'LR', 'BB', 'NC', 'KZ', 'KW', 'JP',
       'GL', 'AI', 'SA', 'BR', 'AU', 'KE', 'IN', 'RS', 'MA', 'MX', 'AR',
       'UY', 'PF', 'CL', 'VC', 'MF', 'SM', 'UA', 'OM', 'CK', 'TN', 'IS',
       'MU', 'CW', 'PE', 'AO', 'VE', 'TC', 'PH', 'ID', 'CO', 'FO', 'VU',
       'JO', 'TW', 'VN', 'EG', 'ZM', 'WS', 'ZW', 'LB', 'TH', 'JM', 'BJ',
       'BF', 'CM', 'CG', 'CI', 'MG', 'ML', 'CF', 'SN', 'TG', 'GH', 'DZ',
       'GN', 'GA', 'LA', 'PK', 'MD', 'QA', 'TZ', 'PG', 'BH', 'TT', 'AM',
       'CR', 'FK', 'BN', 'MN', 'BW', 'MZ', 'GE', 'AD', 'LC', 'LK', 'VI',
       '-1', 'AZ', 'IO', 'MO', 'DJ', 'MK', 'DM', 'A

In [ ]:
#merge panel and gleif data to check on address and name matching
check_df = pd.merge(left=nic_ids, right=gleif_df, on="id_lei", how="left")
display_cols = ["institution_id",
                "id_lei",
                "nm_lgl",
                "legal_name",
                "respondent_city",
                "city",
                "legal_city",
                "hq_city",
                "respondent_state_fips_number",
                "respondent_state_code",  
                "state_cd",
                "street_line1",
                "street_line2",
                "legal_street1",
                "legal_street2",
                "hq_street1",
                "hq_street2",
                "zip_cd",
                "legal_post",
                "legal_region",
                "hq_country",
                "agency_code"]

for col in display_cols:
    if col != "id_lei":
        check_df[col] = check_df[col].apply(lambda x: str(x).lower())

check_df[display_cols]

In [ ]:
print(len(check_df[display_cols][(check_df.nm_lgl==check_df.legal_name)&(~check_df.nm_lgl.isin(["nan", "-1"]))]), "rows matched on name")
check_df[display_cols][(check_df.nm_lgl==check_df.legal_name)&(~check_df.nm_lgl.isin(["nan", "-1"]))]

In [ ]:
print(len(check_df[display_cols][(check_df.respondent_city==check_df.legal_city)&(~check_df.legal_city.isin(["nan","-1"]))]), "rows matched on city")
check_df[display_cols][(check_df.respondent_city==check_df.legal_city)&(~check_df.legal_city.isin(["nan","-1"]))]

In [ ]:
#simple fuzz metrics
check_df['name_simple_fuzz'] = check_df.apply(lambda x: fuzz.ratio(x.nm_lgl, x.legal_name), axis=1)
check_df['city_simple_fuzz'] = check_df.apply(lambda x: fuzz.ratio(x.city, x.legal_city), axis=1)
check_df['street_simple_fuzz'] = check_df.apply(lambda x: fuzz.ratio(x.street_line1, x.legal_street1), axis=1)
#legal_name vs nm_lgl
#city and legal city
#street_line1 vs legal_street1
display_cols.append('name_simple_fuzz')
display_cols.append('city_simple_fuzz')
display_cols.append('street_simple_fuzz')
check_df[display_cols].sort_values(['name_simple_fuzz', 'city_simple_fuzz', 'street_simple_fuzz'], ascending=False)

In [ ]:
#fuzz.partial_ratio("this is a test", "this is a test!")
#fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")
#fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
check_df[display_cols].sort_values(['name_simple_fuzz', 'city_simple_fuzz', 'street_simple_fuzz'], ascending=False)


In [ ]:
print(len(check_df[check_df.name_simple_fuzz>90]), "simple name match")
print(len(check_df[check_df.city_simple_fuzz>90]), "simple city match")
print(len(check_df[check_df.street_simple_fuzz>90]), "simple street match")

In [ ]:
print(len(check_df[(check_df.city_simple_fuzz>90)&(check_df.street_simple_fuzz>90)]), "city and street")

In [ ]:
#pull LEI based on match criteria over 90
    #legal_name vs nm_lgl
    #city and legal city
    #street_line1 vs legal_street1
#put fuzz stuffs
#for row in panel
    #create fuzzy metrics
        #loop over gleif relevant column
        #choose with highest match
    #if fuzzy metrics >= 90
        #copy LEI in from GLEIF
fuzzy_list = []
high_score = 0
for index, row in nic_ids.iterrows():
    print(index)
    for g_index, g_row in gleif_df.iterrows():
        
        city_match_score = fuzz.ratio(row.nm_lgl, g_row.legal_name)
        street_match_score = fuzz.ratio(row.street_line1, g_row.legal_street1)
        name_match_score = fuzz.ratio(row.nm_lgl, g_row.legal_name)
        total_score = city_match_score + street_match_score + name_match_score
        if total_score > high_score:
            high_score = total_score
        #if city_match_score >=85 and street_match_score>=85 and name_match_score>=85:
            #print(city_match_score, "city")
            fuzzy_list.append({
                "inst_id":row.institution_id,
                "nic_name":row.nm_lgl,
                "gleif_name":g_row.legal_name,
                "nic_lei":row.id_lei,
                "gleif_lei":g_row.id_lei,
                "panel_city":row.respondent_city,
                "gleif_city":g_row.legal_city,
                "panel_street":row.street_line1,
                "gleif_street":g_row.legal_street1,
                "city_score":city_match_score,
                "street_score":street_match_score,
                "name_score":name_match_score
            })
    if len(fuzzy_list) >=10:
        break

In [ ]:
gleif_df.columns

In [ ]:
nic_ids.columns